In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
fname = 'yelp_review1'

In [3]:
df = pd.read_csv(f'../raw_data/yelp/{fname}')

In [7]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'star', 'useful', 'funny', 'cool',
       'review', 'date', 'name', 'stars_avg', 'review_count', 'is_open',
       'categories', 'is_Active Life', 'is_Arts & Entertainment',
       'is_Automotive', 'is_Beauty & Spas', 'is_Education',
       'is_Event Planning & Services', 'is_Financial Services', 'is_Food',
       'is_Health & Medical', 'is_Home Services', 'is_Hotels & Travel',
       'is_Local Flavor', 'is_Local Services', 'is_Mass Media', 'is_Nightlife',
       'is_Pets', 'is_Professional Services',
       'is_Public Services & Government', 'is_Real Estate',
       'is_Religious Organizations', 'is_Restaurants', 'is_Shopping'],
      dtype='object')

In [35]:
!python3 -m spacy download en_core_web_sm

2022-02-19 12:40:51.306443: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-19 12:40:51.306512: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
     |████████████████████████████████| 13.9 MB 5.9 MB/s            
You should consider upgrading via the '/home/mathieu/.pyenv/versions/customerchurn/bin/python3 -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [36]:
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector

def get_lang_detector(nlp, name):
    return LanguageDetector()

nlp = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)
text = 'hello word'
doc = nlp(text)
print(doc._.language)

{'language': 'nl', 'score': 0.999993160971745}


In [46]:
sectors = ['Real Estate','Financial Services']
for sector in sectors:
    negative_reviews = []
    positive_reviews = []

    for i in range(1,18):
        fname = f'yelp_review{i}'
        df = pd.read_csv(f'../raw_data/yelp/{fname}')

        mask1 = df[f'is_{sector}'] == 1
        df_temp = df[mask1][['review','star']].copy()
        mask2 = df_temp['star'] < 3
        df_temp_neg = df_temp[mask2][['review']].copy()
        mask3 = df_temp['star'] > 3
        df_temp_pos = df_temp[mask3][['review']].copy()

        negative_reviews.append(df_temp_neg.copy())
        positive_reviews.append(df_temp_pos.copy())
        
    negative_reviews_all =  pd.concat(negative_reviews)
    positive_reviews_all =  pd.concat(positive_reviews)
    
    negative_reviews_all['lang'] = negative_reviews_all['review'].apply(lambda row: nlp(str(row))._.language['language'])
    positive_reviews_all['lang'] = positive_reviews_all['review'].apply(lambda row: nlp(str(row))._.language['language'])
    
    negative_reviews_all [negative_reviews_all['lang'] == 'en'][['review']].to_csv(f'../raw_data/yelp/yelp_{sector}_neg.csv')
    positive_reviews_all [positive_reviews_all['lang'] == 'en'][['review']].to_csv(f'../raw_data/yelp/yelp_{sector}_pos.csv')
        
    

In [31]:
|